# init

In [5]:
import sys
sys.path.append('../../')
sys.path.append('../infras/cellMix/')
sys.path.append('../infras/cytof_data/')
sys.path.append('../infras/')
sys.path.append('../infras/dashboards/')
sys.path.append('../experiments/')
sys.path.append('../experiments/pipeline/')
sys.path.append('../preprocess/intra_variance/')
sys.path.append('../models/cell_proportions/')
sys.path.append('../measures/cell_proportions_measures/')
sys.path.append('../preprocess/cell_specifics/')

In [6]:
from data_factory import DataFactory
from global_utils import GlobalUtils
from cytof_cell_count_infra import CytofCellCountInfra
from cell_proportions_experiments import  CellProportionsExperiments
import exploration_cytof_plots as cytof_plots
from basic import BasicDeconv
from pp_entropy_based import PpEntropyBased
from pp_dep_de_based import  PpDepDeBased
from cell_proportions_measure import CellProportionsMeasure
from pp_clean_high_intra_var import PpCleanHighIntraVar
from pp_clean_irrelevant_proteins import PpCleanIrrelevantProteins
from pp_empty import PpEmpty
from pp_entropy_based_only_largest import PpEntropyBasedOnlyLargest
from aggregate_intra_variance import AggregateIntraVariance
from pipeline_deconv import PipelineDeconv
from deconv_py.preprocess.base import BasePreprocess as PP_base
from deconv_py.preprocess.cell_specific import CellSpecific as PP_proteins

from pp_keep_specific_cells  import  PpKeepSpecificCells
from pp_agg_to_specific_cells import PpAggToSpecificCells

# from deconv_py.infras.data_factory import DataFactory
from deconv_py.infras.data_loader import DataLoader
from deconv_py.models.base import Base as Models_base
from deconv_py.models.cell_proportions_models import CellProportions
from deconv_py.models.cell_specific_models import CellSpecificPerPermutation
from deconv_py.experiments.cell_specific import CellSpecificMetricsPlot
from cellMix_coordinator import CellMixCoordinator

import pandas as pd
import numpy as np
from functools import partial
import multiprocessing
from sklearn import pipeline
import itertools
from scipy.optimize import least_squares
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet

from functools import partial
from scipy.optimize import minimize
import scipy.optimize
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns
import pprint
import os
import pickle as pkl 
from sklearn.decomposition import PCA
from IPython.display import display, HTML


%connect_info

ModuleNotFoundError: No module named 'infras'

# read data

In [ ]:
data_factory = DataFactory()
A_all_vs,B_all_vs = data_factory.load_IBD_all_vs("Intensity",index_func=lambda x:x.split(";")[0],log2_transformation=True)

# basic signature

In [ ]:
agg_iv = AggregateIntraVariance()
#we take from the proteins we target
pre_picked_signature_df =pd.read_csv("../notebooks/targeted_proteins.csv").set_index(["protein","gene"])
pre_picked_signature_df = pre_picked_signature_df[pre_picked_signature_df.source=='enrichment_based genes']
proteins = pre_picked_signature_df.index.intersection(A_all_vs.index)
A_sig,B_sig = agg_iv.transform([A_all_vs.loc[proteins],B_all_vs.loc[proteins]])

B_sig = B_sig.dropna(axis=0)
A_sig = A_sig.loc[B_sig.index]

# nnls

In [ ]:
cell_specific,mass_spec = A_sig,B_sig

cell_abundance_over_samples = []

for sample in mass_spec:
    cell_abundance = scipy.optimize.nnls(cell_specific, mass_spec[sample])[0]
    cell_abundance_df = pd.DataFrame(data=cell_abundance, index=cell_specific.columns, columns=[sample])
    cell_abundance_over_samples.append(cell_abundance_df)

cell_abundance_over_samples_df = pd.concat(cell_abundance_over_samples, axis=1)

cell_abundance_over_samples_df = cell_abundance_over_samples_df / cell_abundance_over_samples_df.sum(
    axis=0)
cell_abundance_over_samples_df = cell_abundance_over_samples_df.round(2)
cell_abundance_over_samples_df

# regression

In [ ]:
cell_specific,mass_spec = A_sig.copy(deep=True),B_sig.copy(deep=True)

cell_abundance_over_samples = []

for sample in mass_spec: 
    regressor = ElasticNet(positive=True,tol=1,alpha=200,normalize=True)
    regressor.fit(cell_specific, mass_spec[sample])
    cell_abundance = regressor.coef_
    cell_abundance_df = pd.DataFrame(data=cell_abundance, index=cell_specific.columns, columns=[sample])
    cell_abundance_over_samples.append(cell_abundance_df)

cell_abundance_over_samples_df = pd.concat(cell_abundance_over_samples, axis=1)

cell_abundance_over_samples_df = cell_abundance_over_samples_df / cell_abundance_over_samples_df.sum(
    axis=0)
cell_abundance_over_samples_df = cell_abundance_over_samples_df.round(2)
cell_abundance_over_samples_df

In [ ]:
ccci = CytofCellCountInfra()
cpm = CellProportionsMeasure()
deconv_result,known_results = ccci.return_mass_and_cytof_not_none_cells_counts(cell_abundance_over_samples_df,filter_by_version="")


if not any([(col in known_results.index.tolist()) for col in deconv_result.index]) :
    columns_mapping = GlobalUtils.get_corospanding_cell_map_from_lists(known_results.index.to_list(),
                                                                           deconv_result.index.to_list())
    known_results = known_results.rename(index=columns_mapping)
    
mixtures_map = GlobalUtils.get_corospanding_mixtures_map_from_lists(deconv_result.columns,known_results.columns)
deconv_result = deconv_result.rename(columns=mixtures_map)


measure_function = lambda x,y : x.corrwith(y,method="spearman", axis=0)
res = measure_function(deconv_result,known_results)

res.mean()

In [ ]:
res.hist()